In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import imageio
import time
from IPython import display
from skimage.io import imread
import os
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
import pickle
from tensorflow.keras import *
import os
from tensorflow.keras.regularizers import *
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.layers import *
import numpy as np
import PIL
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import scipy
import imageio
from PIL import Image,ImageOps
from sklearn.model_selection import train_test_split
from scipy import ndimage
from tensorflow.keras.utils import to_categorical
from scipy.io import loadmat
import h5py
from skimage.color import rgb2gray
import cv2
from tensorflow.keras.models import model_from_json
import pandas as pd
from tensorflow.keras.optimizers import *
from skimage.color import gray2rgb
from IPython.display import FileLink
%matplotlib inline

# Setting all seed values to zero for similar results
seed=0

from numpy.random import seed
seed(0)
tf.random.set_seed(0)

In [ ]:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, 
        rotation_range=360,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.5,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=0.2, 
    )

    train_generator = train_datagen.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/train',
        target_size=(96, 96),
        color_mode='rgb',
        class_mode="categorical",
        subset='training',
    )
    validation_generator = train_datagen.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/train',
        target_size=(96,96),
        color_mode='rgb',
        class_mode="categorical",
        subset='validation',
    )

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/',
        classes=['test'],
        target_size=(96, 96),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical')


In [ ]:
train_generator.class_indices

In [ ]:
class_labels = list(train_generator.class_indices)
class_labels

In [ ]:
plt.imshow(train_generator[0][0][1])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_generator[0][0][i])
    plt.xlabel(class_labels[np.argmax(train_generator[0][1][i])])
plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.applications.vgg19 import VGG19
model = VGG19(input_shape=(96, 96, 3), include_top=False,  weights="imagenet")
model.summary()

In [ ]:
for ix in range(len(model.layers)):
    print(ix, model.layers[ix])

In [ ]:
for ix in range(19):
    model.layers[ix].trainable = False 


In [ ]:
model.summary()

In [ ]:
model = tf.keras.Sequential([
    model,
    GlobalAveragePooling2D(),
    Dense(100, activation="relu"),
    BatchNormalization(trainable = True,axis=1),
    
    Dropout(0.5),
    
    Dense(50, activation="relu"),
    BatchNormalization(trainable = True,axis=1),
    
    Dense(12,activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
for ix in range(len(model.layers)):
    print(ix, model.layers[ix])


In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20,)
model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=[f1_m])

In [ ]:
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[es])

In [ ]:
predictions = model.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predictions.shape[0]):
  y_class = predictions[i, :].argmax(axis=-1)
  class_list += [class_labels[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head(20)